In [1]:
import numpy as np

In [2]:
task_order = np.zeros(3030, dtype=int)
perm = [0, 2, 1]

In [3]:
ext_ = ''.join(map(str,perm))

In [4]:
ext_

'021'

In [5]:
task_order[:1010] = perm[0]
task_order[1010:2020] = perm[1]
task_order[2020:] = perm[2]

In [6]:
task_order

array([0, 0, 0, ..., 1, 1, 1])

In [7]:
np.save('task-order-{}.npy'.format(ext_), task_order)